# MCPResource in OpenDXA

This tutorial covers the Model Context Protocol (MCP) resource in OpenDXA, which provides a standardized way to integrate external services and tools into your agents.

## Learning Objectives

By the end of this tutorial, you will understand:

1. What MCP is and how it works
2. How to create and use MCP resources
3. How to work with different transport types (STDIO and HTTP)
4. How to discover and use MCP tools
5. Best practices for MCP resource usage

## Prerequisites

- Basic understanding of OpenDXA's architecture
- Familiarity with Python async/await syntax
- Understanding of basic resource management concepts

## 1. Understanding MCP

The Model Context Protocol (MCP) is a standardized way to expose data and functionality to LLM applications. MCP servers can:

1. **Expose Data**: Through resources (similar to GET endpoints)
2. **Provide Functionality**: Through tools (similar to POST endpoints)
3. **Define Interaction Patterns**: Through prompts (reusable templates)

Let's start by creating a simple MCP resource:

In [ ]:
from opendxa.common.resource.mcp import McpResource, StdioTransportParams
from typing import Dict, Any

# Create a local MCP resource using STDIO transport
echo = McpResource(
    name="echo",
    transport_params=StdioTransportParams(
        server_script="examples/learning_paths/02_core_concepts/mcp_servers/mcp_echo.py",
        command="python",
        args=["examples/learning_paths/02_core_concepts/mcp_servers/mcp_echo.py"]
    )
)

# Test the echo server
response = await echo.query({
    "tool": "echo",
    "arguments": {"message": "Hello from MCP!"}
})

print(f"Echo response: {response.content}")

## 2. Transport Types

MCP resources support two types of transport:

1. **STDIO Transport**: For local MCP servers
2. **HTTP Transport**: For remote MCP servers using Server-Sent Events (SSE)

Let's see how to use both:

In [ ]:
from opendxa.common.resource.mcp import HttpTransportParams

# Example 1: STDIO Transport (Local Server)
local_mcp = McpResource(
    name="local_echo",
    transport_params=StdioTransportParams(
        server_script="examples/learning_paths/02_core_concepts/mcp_servers/mcp_echo.py",
        command="python",
        args=["examples/learning_paths/02_core_concepts/mcp_servers/mcp_echo.py"],
        env={"DEBUG": "1"}  # Optional environment variables
    )
)

# Example 2: HTTP Transport (Remote Server)
remote_mcp = McpResource(
    name="remote_echo",
    transport_params=HttpTransportParams(
        url="https://api.example.com/mcp",
        headers={"Authorization": "Bearer your-token"},
        timeout=5.0,  # Connection timeout in seconds
        sse_read_timeout=300.0  # SSE read timeout in seconds
    )
)

# Test both servers
local_response = await local_mcp.query({
    "tool": "ping"
})
print(f"Local server response: {local_response.content}")

try:
    remote_response = await remote_mcp.query({
        "tool": "ping"
    })
    print(f"Remote server response: {remote_response.content}")
except Exception as e:
    print(f"Remote server error: {e}")

## 3. Tool Discovery and Usage

MCP resources provide a way to discover available tools at runtime. Let's see how to use this feature:

In [ ]:
# Discover available tools
tools = await local_mcp.list_tools()
print(f"Found {len(tools)} available tools\n")

# Print tool details
for tool in tools:
    print(f"Tool: {tool.name}")
    print(f"Description: {tool.description}")
    print("Parameters:")
    for param_name, param_details in tool.inputSchema["properties"].items():
        print(f"  - {param_name}: {param_details.get('type')}")
        if param_name in tool.inputSchema.get("required", []):
            print("    (Required)")
    print()

# Example: Using a discovered tool
if tools:
    tool = tools[0]  # Use the first available tool
    print(f"Testing tool: {tool.name}")
    
    # Prepare arguments based on the tool's schema
    arguments = {}
    for param_name, param_details in tool.inputSchema["properties"].items():
        if param_name in tool.inputSchema.get("required", []):
            # Provide a default value based on the parameter type
            param_type = param_details.get("type")
            if param_type == "string":
                arguments[param_name] = "test"
            elif param_type == "number":
                arguments[param_name] = 42
            elif param_type == "boolean":
                arguments[param_name] = True
            elif param_type == "array":
                arguments[param_name] = []
            elif param_type == "object":
                arguments[param_name] = {}
    
    # Execute the tool
    response = await local_mcp.query({
        "tool": tool.name,
        "arguments": arguments
    })
    
    print(f"Tool response: {response.content}")

## 4. Error Handling

MCP resources provide robust error handling. Let's see how to handle different types of errors:

In [ ]:
# Example 1: Invalid tool name
try:
    response = await local_mcp.query({
        "tool": "nonexistent_tool",
        "arguments": {}
    })
    print(f"Response: {response.content}")
except Exception as e:
    print(f"Error: {e}")

# Example 2: Invalid arguments
try:
    response = await local_mcp.query({
        "tool": "echo",
        "arguments": {"invalid_param": "value"}
    })
    print(f"Response: {response.content}")
except Exception as e:
    print(f"Error: {e}")

# Example 3: Missing required arguments
try:
    response = await local_mcp.query({
        "tool": "echo"  # Missing required 'message' argument
    })
    print(f"Response: {response.content}")
except Exception as e:
    print(f"Error: {e}")

## 5. Advanced Features

### 5.1 Environment Variables

You can pass environment variables to local MCP servers:

In [ ]:
# Create MCP resource with environment variables
mcp_with_env = McpResource(
    name="env_mcp",
    transport_params=StdioTransportParams(
        server_script="examples/learning_paths/02_core_concepts/mcp_servers/mcp_echo.py",
        command="python",
        args=["examples/learning_paths/02_core_concepts/mcp_servers/mcp_echo.py"],
        env={
            "DEBUG": "1",
            "LOG_LEVEL": "INFO",
            "CUSTOM_VAR": "custom_value"
        }
    )
)

# Test the server with environment variables
response = await mcp_with_env.query({
    "tool": "ping"
})
print(f"Response: {response.content}")

### 5.2 Tool Schema Validation

MCP automatically validates tool arguments against their schemas:

In [ ]:
# Example 1: Valid arguments
try:
    response = await local_mcp.query({
        "tool": "echo",
        "arguments": {"message": "Valid message"}
    })
    print(f"Valid arguments response: {response.content}")
except Exception as e:
    print(f"Error: {e}")

# Example 2: Invalid argument type
try:
    response = await local_mcp.query({
        "tool": "echo",
        "arguments": {"message": 42}  # Should be a string
    })
    print(f"Response: {response.content}")
except Exception as e:
    print(f"Error: {e}")

## 6. Best Practices

Here are some best practices for working with MCP resources:

1. **Server Design**
   - Keep servers focused on specific functionality
   - Implement proper error handling
   - Use type hints and docstrings
   - Follow the single responsibility principle

2. **Client Usage**
   - Always initialize resources before use
   - Implement proper error handling
   - Use appropriate timeouts for remote servers
   - Validate tool arguments before calling

3. **Transport Selection**
   - Use STDIO transport for local servers
   - Use HTTP transport for remote servers
   - Configure appropriate timeouts
   - Handle connection errors gracefully

4. **Tool Design**
   - Keep tools focused and single-purpose
   - Provide clear documentation
   - Use appropriate parameter types
   - Handle edge cases

5. **Error Handling**
   - Implement proper error handling
   - Use appropriate error messages
   - Handle timeouts and connection errors
   - Validate inputs and outputs

## Summary

In this tutorial, we covered:

1. Understanding MCP and its features
2. Working with different transport types
3. Discovering and using MCP tools
4. Handling errors and edge cases
5. Using advanced features
6. Following best practices

The MCP resource in OpenDXA provides a powerful and flexible way to integrate external services and tools into your agents. By following the best practices outlined in this tutorial, you can create robust and maintainable MCP-based applications.